In [ ]:
!pip install -q openai-whisper

!pip install -q chromadb
!pip install -q langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 14.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s

In [ ]:

!pip install -q moviepy
!pip install -q sentence_transformers

In [ ]:
!pip install -q opencv-python




In [ ]:
import cv2
import os
import whisper as wh
# import speech_recognition as sr
from moviepy.editor import VideoFileClip
import numpy as np
from transformers import CLIPProcessor, CLIPModel
from sentence_transformers import SentenceTransformer
import torch
import numpy as np
from PIL import Image
import chromadb

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def process_video(video_path):
    """Process the video to extract frames and audio"""
    # Create directories if they don't exist
    os.makedirs("frames", exist_ok=True)
    os.makedirs("audio", exist_ok=True)
      # Extract frames
    frames = extract_frames(video_path) #############################
    # Extract audio
    audio_path = extract_audio(video_path)

    # Convert audio to text
    transcript = audio_to_text(audio_path)

    # Generate embeddings
    frame_embeddings = generate_frame_embeddings(frames) ###############################
    text_embedding = generate_text_embeddings(transcript)

    # Store embeddings in ChromaDB
    store_embeddings(frame_embeddings, text_embedding, transcript)#########################################


    return True

def extract_frames(video_path, frame_rate=1):
    """Extract frames from the video at the specified frame rate"""
    frames = []
    frame_paths = []

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = 0

    while True:
        read, frame = cap.read()  #read is boolean returned by cap.read(): true if frame is read otherwise false. frame is actual frame
        if not read:
            break

        # Save frame at specified frame rate
        if frame_count % int(fps / frame_rate) == 0:
            frame_path = f"/content/frames/frame_{frame_count}.jpg"
            cv2.imwrite(frame_path, frame)
            frames.append(frame)
            frame_paths.append(frame_path)

        frame_count += 1

    cap.release()
    return frame_paths

def extract_audio(video_path):
    """Extract audio from the video"""
    audio_path = "/content/audio/extracted_audio.wav"
    video = VideoFileClip(video_path)
    video.audio.write_audiofile(audio_path)
    return audio_path

def audio_to_text(audio_path):
    """Convert audio to text using speech recognition"""
    # recognizer = sr.Recognizer()

    # with sr.AudioFile(audio_path) as source:
    model= wh.load_model("base")
        # audio_data = recognizer.record(source)
        # text = recognizer.recognize_google(audio_data)
    result=model.transcribe(audio_path)
    text=result["text"]

    # Save transcript
    with open("transcript.txt", "w") as f:
        f.write(text)

    # return text
    loader=TextLoader("/content/transcript.txt")
    text_doc=loader.load()
    return text_doc


def generate_frame_embeddings(frame_paths):
    """Generate embeddings for video frames using CLIP"""
    # Load CLIP model
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

    frame_embeddings = []

    for frame_path in frame_paths:
        image = Image.open(frame_path)
        image = image.resize((224, 224))
        inputs = processor(images=image, return_tensors="pt", padding=True, truncation=True) #pyTorch #inputs is a dictionary
        with torch.no_grad(): #: This is used to perform inference without calculating gradients, making it faster and reducing memory usage since we aren't updating the model parameters.
            image_features = model.get_image_features(**inputs)
            # print(f"Frame embedding dimension: {image_features.shape}")
        frame_embeddings.append({
            "path": frame_path,
            "embedding": image_features.squeeze().cpu().numpy()
        })

    return frame_embeddings


def generate_text_embeddings(text_doc):

    """Generate embeddings for text using CLIPProcessor and CLIPModel"""

    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

    text_splitter= RecursiveCharacterTextSplitter(chunk_size=77, chunk_overlap=10)
    documents=text_splitter.split_documents(text_doc)
    # print(documents)
    # Process the text input
    text_embeddings_list = []
    for i, document in enumerate(documents):
        text=document.page_content
        inputs = processor(text=[text], return_tensors="pt", padding=True, truncation=True)

       # Get text embeddings from the model
        with torch.no_grad():
           text_embeddings = model.get_text_features(**inputs)
           text_id = f"text_{i}"
        text_embeddings_list.append( {
            "id" : text_id,
            "text": text,
            "embedding": text_embeddings.squeeze().cpu().numpy()
        })

    return text_embeddings_list


def store_embeddings(frame_embeddings,text_embedding, transcript):
    """Store embeddings in ChromaDB"""
    client = chromadb.Client()

    # Create collections
    frames_collection = client.get_or_create_collection(name="video_frames") #######################
    text_collection = client.get_or_create_collection(name="video_text")
    # client = chromadb.PersistentClient()

    # # Recreate collections with the correct dimensionality (512)
    # frames_collection = recreate_collection(client, "video_frames", 512)
    # text_collection = recreate_collection(client, "video_text", 512)

    # Add frame embeddings
    for i, frame in enumerate(frame_embeddings):###############################
        frames_collection.add(
            embeddings=[frame["embedding"].tolist()],
            documents=[f"Frame {i}"],
            metadatas=[{"path": frame["path"]}],
            ids=[f"frame_{i}"]
        )  ############################

    # Add text embedding
    for i, text in enumerate(text_embedding):
      text_collection.add(
          embeddings=[text["embedding"].tolist()],
          documents=[text["text"]],
          metadatas=[{"type": "transcript"}],
          ids=[text["id"]]
    )



In [ ]:
process_video("/content/temp_video.mp4")

MoviePy - Writing audio in /content/audio/extracted_audio.wav


MoviePy - Done.


100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 55.4MiB/s]
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

True

In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import BlipProcessor, BlipForConditionalGeneration

import chromadb
def search_relevant_content(query):
    """Search for relevant content based on the query"""
    # Generate embedding for the query
    model = SentenceTransformer('clip-ViT-B-32')
    query_embedding = model.encode(query)

    tokenizer=BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    cap_model=BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
    # Search in ChromaDB
    client = chromadb.Client()
    # retriever= client.as_retriever()
    frames_collection = client.get_collection(name="video_frames")
    text_collection = client.get_collection(name="video_text")


    # text_retriever= Chroma(collection=text_collection, embedding_function=)

    # Search for relevant frames
    frame_results = frames_collection.query(
        query_embeddings=[query_embedding.tolist()],
        n_results=15,
        include=["documents", "metadatas"]
    )
    caption = ""
    for metadata in frame_results["metadatas"][0]:
        frame_path = metadata["path"]  # Get the image path from metadata
        image = Image.open(frame_path).convert("RGB")  # Open and preprocess the image

        inputs = tokenizer(images=image, return_tensors="pt")
        captions = cap_model.generate(**inputs)
        caption_text = tokenizer.decode(captions[0], skip_special_tokens=True)

        caption += f"Frame content: {caption_text})\n"

    text_results = text_collection.query(
        query_embeddings=[query_embedding.tolist()],
        n_results=15
    )
    print("test results : ", text_results)
    # Combine results:
    content = ""

    for document in text_results["documents"][0]:

        content += f"Video transcript: {document}\n\n"

    # for document, metadata in zip(frame_results["documents"][0], frame_results["metadatas"][0]):
    #     # print("##############################", type(document), ":  ", document, "######################################################################################")
    #     content += f"Frame content: {document} (Path: {metadata['path']})\n"
    content += caption
    return content

In [ ]:
query= search_relevant_content("why do we use cosmosDB")

test results :  {'ids': [['text_71', 'text_92', 'text_98', 'text_15', 'text_94', 'text_76', 'text_49', 'text_78', 'text_5', 'text_99', 'text_59', 'text_0', 'text_72', 'text_50', 'text_16']], 'embeddings': None, 'documents': [['scale and sharding are built into Cosmos DB by default. That is the actual', 'existing applications to work with Cosmos DB. For instance, if I had written', 'to work with Cosmos DB. This is understanding how for a unstructured data', "DB. Cosmos DB looks and feels a lot like MongoDB if you've ever used", 'quickly repurpose that to work with Cosmos DB by creating the resource right', 'content on CBT Nuggets to dig deep into the actual architecture of Cosmos DB', 'in the types of data that we store. Now the big wins for Cosmos DB is the', 'I could deploy Cosmos DB and have it replicate across these different data', "in this video, we're going to take a look at how Cosmos DB really works and", 'data source like Cosmos DB, which could be relational, maybe or maybe no

In [ ]:
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate

from huggingface_hub import login
from google.colab import userdata

def generate_response(query, content):
    """Generate a response based on the query"""

    relevant_content = content

    #logging in to hugging face to use the model
    logging_token=userdata.get('HUGGING_FACE_TOKEN')
    login(token = logging_token )

    llm = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-Instruct-v0.2",
    task="text-generation",
    model_kwargs={"temperature": 0.7, "torch_dtype": "auto"},
    pipeline_kwargs={
        "max_new_tokens": 512,
    },
    device=0
)

    # Create a more direct prompt
    prompt = PromptTemplate.from_template("""
      I have Extracted information from this video content and answer the question based on what you think the video is all about.

      VIDEO CONTEXT:
      {context}

      QUESTION: {input}

      ANSWER:
       """)

    output_parser=StrOutputParser()
    chain=prompt|llm|output_parser

    response = chain.invoke({"input":query, "context":relevant_content})
    return response


In [ ]:

res = generate_response("why do we use cosmosDB", query)
print(res.split("ANSWER:")[1].strip())

  warnings.warn(

  warnings.warn(



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


Cosmos DB is a globally distributed, multi-model database service that offers turnkey
        global distribution and horizontal scale with guaranteed strong consistency. It is designed
        to handle various types of data and is geographically replicated. The video suggests that
        Cosmos DB can be used as a replacement for existing databases like MongoDB, and it can be
        easily integrated with existing applications. The big wins for Cosmos DB are its ability to
        handle geographic replication and horizontal scale. It is also mentioned that Cosmos DB
        looks and feels a lot like MongoDB, making it easy for users who have experience with MongoDB
        to quickly repurpose their applications to work with Cosmos DB.


In [ ]:
def beautify_llm_response(res):
    try:
        # Extract the response after "ANSWER:"
        extracted_answer = res.split("ANSWER:")[1].strip()

        # Clean and format the response
        formatted_response = f"\n{'='*40}\n  LLM Response\n{'='*40}\n\n{extracted_answer}\n\n{'='*40}"
        return formatted_response
    except IndexError:
        return "Error: 'ANSWER:' not found in the response."


In [ ]:
print(beautify_llm_response(res))


  LLM Response

Cosmos DB is a globally distributed, multi-model database service that offers turnkey
        global distribution and horizontal scale with guaranteed strong consistency. It is designed
        to handle various types of data and is geographically replicated. The video suggests that
        Cosmos DB can be used as a replacement for existing databases like MongoDB, and it can be
        easily integrated with existing applications. The big wins for Cosmos DB are its ability to
        handle geographic replication and horizontal scale. It is also mentioned that Cosmos DB
        looks and feels a lot like MongoDB, making it easy for users who have experience with MongoDB
        to quickly repurpose their applications to work with Cosmos DB.

